<h1><center>CDS503 Project Data Proprocessing</center></h1>

## Contents
[**A. Movies Metadata**](#mm)  
* [Clean Movies Metadata](#cmm)
* [Merge with Credits Data](#mwcd)
* [Merge with Links Data](#mwld)
* [One Hot Encoding Movies Metadata](#ohemm)

[**B. Ratings Data**](#rd)  
* [Clean Ratings Data](#crd)
* [All Ratings Data](#ard)
* [Ratings Data Subsets](#rds)

## A. Movies Metadata<a id="mm"></a>

*italicized text*### Clean Movies Metadata<a id="cmm"></a>

In [ ]:
# Load movies metadata.
import pandas as pd
pd.set_option('display.max_columns', None)

movies_df = pd.read_csv('movies_metadata.csv')
print(movies_df.shape)
movies_df.head()

(45466, 24)


/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3155: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [ ]:
# Extract movies with 'Released' status, runtime between 60 and 200 minutes, vote score at least 6 (out of 10),
# and at least 20 votes from users.
movies_df = movies_df[(movies_df.status == 'Released') & (movies_df.vote_average >= 6) &
                      (movies_df.runtime >= 60) & (movies_df.runtime <= 200) & (movies_df.vote_count >= 20)]
movies_df.shape

(9751, 24)

In [ ]:
# Convert the values of spoken languages into 'English' or 'Others'.
import ast

temp = []
for instance in movies_df.spoken_languages:
    languages = ast.literal_eval(instance)
    if any([element['name'] == 'English' for element in languages]):
        temp.append('English')
    else:
        temp.append('Others')

# Extract English movies only.
movies_df.spoken_languages = temp
movies_df = movies_df[movies_df.spoken_languages == 'English']
movies_df.shape

(7330, 24)

In [ ]:
# Drop the irrelevant attributes.
movies_df.drop(['adult', 'budget', 'homepage', 'imdb_id', 'original_title', 'overview', 'poster_path', 'revenue',
                'tagline', 'video', 'status', 'production_countries', 'original_language', 'spoken_languages'],
               axis=1, inplace=True)
print(movies_df.shape)
movies_df.head()

(7330, 10)


,belongs_to_collection,genres,id,popularity,production_companies,release_date,runtime,title,vote_average,vote_count
0,"{'id': 10194, 'name': 'Toy Story Collection', ...","[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,21.9469,"[{'name': 'Pixar Animation Studios', 'id': 3}]",1995-10-30,81.0,Toy Story,7.7,5415.0
1,NaN,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,17.0155,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...",1995-12-15,104.0,Jumanji,6.9,2413.0
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...","[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...",1995-12-22,101.0,Grumpier Old Men,6.5,92.0
3,NaN,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,3.85949,[{'name': 'Twentieth Century Fox Film Corporat...,1995-12-22,127.0,Waiting to Exhale,6.1,34.0
5,NaN,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",949,17.9249,"[{'name': 'Regency Enterprises', 'id': 508}, {...",1995-12-15,170.0,Heat,7.7,1886.0


In [ ]:
# Convert some attributes into correct datatypes.
movies_df.id = movies_df.id.astype(int)
movies_df.popularity = movies_df.popularity.astype(float)
movies_df.release_date = pd.to_datetime(movies_df.release_date)
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7330 entries, 0 to 45420
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   belongs_to_collection  1177 non-null   object        
 1   genres                 7330 non-null   object        
 2   id                     7330 non-null   int64         
 3   popularity             7330 non-null   float64       
 4   production_companies   7330 non-null   object        
 5   release_date           7329 non-null   datetime64[ns]
 6   runtime                7330 non-null   float64       
 7   title                  7330 non-null   object        
 8   vote_average           7330 non-null   float64       
 9   vote_count             7330 non-null   float64       
dtypes: datetime64[ns](1), float64(4), int64(1), object(4)
memory usage: 629.9+ KB


In [ ]:
# Drop the instances with duplicated ID.
movies_df.sort_values(by='id', inplace=True)
movies_df.drop(movies_df[movies_df.id.duplicated(keep=False)].index[0::2], axis=0, inplace=True)

# Fill the null values of belongs_to_collection with '[]'.
movies_df.belongs_to_collection.fillna('[]', inplace=True)

# Drop all null values and duplicates.
movies_df.dropna(inplace=True)
movies_df.drop_duplicates(inplace=True)
print(movies_df.shape)
movies_df.head()

(7323, 10)


,belongs_to_collection,genres,id,popularity,production_companies,release_date,runtime,title,vote_average,vote_count
12947,[],"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",3,2.292110,"[{'name': 'Villealfa Filmproduction Oy', 'id':...",1986-10-16,76.0,Shadows in Paradise,7.1,35.0
17,[],"[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name...",5,9.026586,"[{'name': 'Miramax Films', 'id': 14}, {'name':...",1995-12-09,98.0,Four Rooms,6.5,539.0
474,[],"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",6,5.538671,"[{'name': 'Universal Pictures', 'id': 33}, {'n...",1993-10-15,110.0,Judgment Night,6.4,79.0
256,"{'id': 10, 'name': 'Star Wars Collection', 'po...","[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",11,42.149697,"[{'name': 'Lucasfilm', 'id': 1}, {'name': 'Twe...",1977-05-25,121.0,Star Wars,8.1,6778.0
6232,"{'id': 137697, 'name': 'Finding Nemo Collectio...","[{'id': 16, 'name': 'Animation'}, {'id': 10751...",12,25.497794,"[{'name': 'Pixar Animation Studios', 'id': 3}]",2003-05-30,100.0,Finding Nemo,7.6,6292.0


In [ ]:
# Extract collections from json format.
temp = []
for instance in movies_df.belongs_to_collection:
    collections = ast.literal_eval(instance)
    if len(collections) < 1:
        temp.append(None)
    else:
        temp.append(collections['name'])
movies_df.belongs_to_collection = temp
print(movies_df.shape)
movies_df.head()

(7323, 10)


,belongs_to_collection,genres,id,popularity,production_companies,release_date,runtime,title,vote_average,vote_count
12947,None,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",3,2.292110,"[{'name': 'Villealfa Filmproduction Oy', 'id':...",1986-10-16,76.0,Shadows in Paradise,7.1,35.0
17,None,"[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name...",5,9.026586,"[{'name': 'Miramax Films', 'id': 14}, {'name':...",1995-12-09,98.0,Four Rooms,6.5,539.0
474,None,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",6,5.538671,"[{'name': 'Universal Pictures', 'id': 33}, {'n...",1993-10-15,110.0,Judgment Night,6.4,79.0
256,Star Wars Collection,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",11,42.149697,"[{'name': 'Lucasfilm', 'id': 1}, {'name': 'Twe...",1977-05-25,121.0,Star Wars,8.1,6778.0
6232,Finding Nemo Collection,"[{'id': 16, 'name': 'Animation'}, {'id': 10751...",12,25.497794,"[{'name': 'Pixar Animation Studios', 'id': 3}]",2003-05-30,100.0,Finding Nemo,7.6,6292.0


In [ ]:
# Extract genres from json format.
movies_df.drop(movies_df[movies_df.genres == '[]'].index, inplace=True)
movies_df.sort_values(by='id', ignore_index=True, inplace=True)
temp = []
for i in range(movies_df.shape[0]):
    genre = ast.literal_eval(movies_df.genres[i])
    temp_1 = []
    for element in genre:
        temp_1.append(element['name'])
    temp.append(temp_1)
movies_df.genres = temp
print(movies_df.shape)
movies_df.head()

(7319, 10)


,belongs_to_collection,genres,id,popularity,production_companies,release_date,runtime,title,vote_average,vote_count
0,None,"[Drama, Comedy]",3,2.292110,"[{'name': 'Villealfa Filmproduction Oy', 'id':...",1986-10-16,76.0,Shadows in Paradise,7.1,35.0
1,None,"[Crime, Comedy]",5,9.026586,"[{'name': 'Miramax Films', 'id': 14}, {'name':...",1995-12-09,98.0,Four Rooms,6.5,539.0
2,None,"[Action, Thriller, Crime]",6,5.538671,"[{'name': 'Universal Pictures', 'id': 33}, {'n...",1993-10-15,110.0,Judgment Night,6.4,79.0
3,Star Wars Collection,"[Adventure, Action, Science Fiction]",11,42.149697,"[{'name': 'Lucasfilm', 'id': 1}, {'name': 'Twe...",1977-05-25,121.0,Star Wars,8.1,6778.0
4,Finding Nemo Collection,"[Animation, Family]",12,25.497794,"[{'name': 'Pixar Animation Studios', 'id': 3}]",2003-05-30,100.0,Finding Nemo,7.6,6292.0


In [ ]:
# Extract production companies from json format.
movies_df.drop(movies_df[movies_df.production_companies == '[]'].index, inplace=True)
movies_df.reset_index(drop=True, inplace=True)
temp = []
for i in range(movies_df.shape[0]):
    company = ast.literal_eval(movies_df.production_companies[i])
    temp_1 = []
    for element in company:
        temp_1.append(element['name'])
    temp.append(temp_1)
movies_df.production_companies = temp
print(movies_df.shape)
movies_df.head()

(6945, 10)


,belongs_to_collection,genres,id,popularity,production_companies,release_date,runtime,title,vote_average,vote_count
0,None,"[Drama, Comedy]",3,2.292110,[Villealfa Filmproduction Oy],1986-10-16,76.0,Shadows in Paradise,7.1,35.0
1,None,"[Crime, Comedy]",5,9.026586,"[Miramax Films, A Band Apart]",1995-12-09,98.0,Four Rooms,6.5,539.0
2,None,"[Action, Thriller, Crime]",6,5.538671,"[Universal Pictures, Largo Entertainment, JVC ...",1993-10-15,110.0,Judgment Night,6.4,79.0
3,Star Wars Collection,"[Adventure, Action, Science Fiction]",11,42.149697,"[Lucasfilm, Twentieth Century Fox Film Corpora...",1977-05-25,121.0,Star Wars,8.1,6778.0
4,Finding Nemo Collection,"[Animation, Family]",12,25.497794,[Pixar Animation Studios],2003-05-30,100.0,Finding Nemo,7.6,6292.0


### Merge with Credits Data<a id="mwcd"></a>

In [ ]:
# Load credits data.
credits_df = pd.read_csv('credits.csv')
print(credits_df.shape)
credits_df.head()

(45476, 3)


,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [ ]:
# Extract credits based on the movies left in the movies_df.
credits_df = credits_df[credits_df.id.isin(movies_df.id)]
credits_df = credits_df.drop_duplicates()
credits_df.sort_values('id', inplace=True)
credits_df.drop(credits_df[credits_df.id.duplicated(keep=False)].index[0::2], axis=0, inplace=True)
credits_df.reset_index(drop=True, inplace=True)
print(credits_df.shape)
credits_df.head()

(6945, 3)


,cast,crew,id
0,"[{'cast_id': 5, 'character': 'Nikander', 'cred...","[{'credit_id': '52fe420dc3a36847f8000077', 'de...",3
1,"[{'cast_id': 42, 'character': 'Ted the Bellhop...","[{'credit_id': '52fe420dc3a36847f800011b', 'de...",5
2,"[{'cast_id': 7, 'character': 'Frank Wyatt', 'c...","[{'credit_id': '52fe420dc3a36847f800023d', 'de...",6
3,"[{'cast_id': 3, 'character': 'Luke Skywalker',...","[{'credit_id': '52fe420dc3a36847f8000437', 'de...",11
4,"[{'cast_id': 8, 'character': 'Marlin (voice)',...","[{'credit_id': '52fe420ec3a36847f80006b1', 'de...",12


In [ ]:
# Extract directors from json format.
movies_df.drop(credits_df[credits_df.crew == '[]'].index, inplace=True)
movies_df.reset_index(drop=True, inplace=True)
credits_df.drop(credits_df[credits_df.crew == '[]'].index, inplace=True)
credits_df.reset_index(drop=True, inplace=True)
temp = []
for i in range(len(credits_df)):
    temp_1 = ast.literal_eval(credits_df.crew[i])
    temp_2 = []
    for element in temp_1:
        if element['job'] == 'Director':
            temp_2.append(element['name'])
    if len(temp_2) < 1:
        temp.append(None)
    else:
        temp.append(temp_2)

# Merge directors attribute with movies_df.
movies_df['directors'] = temp
print(movies_df.shape)
movies_df.head()

(6940, 11)


,belongs_to_collection,genres,id,popularity,production_companies,release_date,runtime,title,vote_average,vote_count,directors
0,None,"[Drama, Comedy]",3,2.292110,[Villealfa Filmproduction Oy],1986-10-16,76.0,Shadows in Paradise,7.1,35.0,[Aki Kaurismäki]
1,None,"[Crime, Comedy]",5,9.026586,"[Miramax Films, A Band Apart]",1995-12-09,98.0,Four Rooms,6.5,539.0,"[Allison Anders, Alexandre Rockwell, Robert Ro..."
2,None,"[Action, Thriller, Crime]",6,5.538671,"[Universal Pictures, Largo Entertainment, JVC ...",1993-10-15,110.0,Judgment Night,6.4,79.0,[Stephen Hopkins]
3,Star Wars Collection,"[Adventure, Action, Science Fiction]",11,42.149697,"[Lucasfilm, Twentieth Century Fox Film Corpora...",1977-05-25,121.0,Star Wars,8.1,6778.0,[George Lucas]
4,Finding Nemo Collection,"[Animation, Family]",12,25.497794,[Pixar Animation Studios],2003-05-30,100.0,Finding Nemo,7.6,6292.0,[Andrew Stanton]


In [ ]:
# Extract 3 main characters from json format.
movies_df.drop(credits_df[credits_df.cast == '[]'].index, inplace=True)
movies_df.reset_index(drop=True, inplace=True)
credits_df.drop(credits_df[credits_df.cast == '[]'].index, inplace=True)
credits_df.reset_index(drop=True, inplace=True)
temp = []
for i in range(len(credits_df)):
    temp_1 = ast.literal_eval(credits_df.cast[i])
    temp_2 = []
    for element in temp_1:
        if 'voice' in element['character']:
            break
        elif element['order'] < 3:
            temp_2.append(element['name'])
    if len(temp_2) < 1:
        temp.append(None)
    else:
        temp.append(temp_2)

# Merge characters attribute with movies_df.
movies_df['characters'] = temp
print(movies_df.shape)
movies_df.head()

(6921, 12)


,belongs_to_collection,genres,id,popularity,production_companies,release_date,runtime,title,vote_average,vote_count,directors,characters
0,None,"[Drama, Comedy]",3,2.292110,[Villealfa Filmproduction Oy],1986-10-16,76.0,Shadows in Paradise,7.1,35.0,[Aki Kaurismäki],"[Matti Pellonpää, Kati Outinen, Sakari Kuosmanen]"
1,None,"[Crime, Comedy]",5,9.026586,"[Miramax Films, A Band Apart]",1995-12-09,98.0,Four Rooms,6.5,539.0,"[Allison Anders, Alexandre Rockwell, Robert Ro...","[Tim Roth, Antonio Banderas, Jennifer Beals]"
2,None,"[Action, Thriller, Crime]",6,5.538671,"[Universal Pictures, Largo Entertainment, JVC ...",1993-10-15,110.0,Judgment Night,6.4,79.0,[Stephen Hopkins],"[Emilio Estevez, Cuba Gooding Jr., Denis Leary]"
3,Star Wars Collection,"[Adventure, Action, Science Fiction]",11,42.149697,"[Lucasfilm, Twentieth Century Fox Film Corpora...",1977-05-25,121.0,Star Wars,8.1,6778.0,[George Lucas],"[Mark Hamill, Harrison Ford, Carrie Fisher]"
4,Finding Nemo Collection,"[Animation, Family]",12,25.497794,[Pixar Animation Studios],2003-05-30,100.0,Finding Nemo,7.6,6292.0,[Andrew Stanton],None


In [ ]:
# Drop movies that do not have director.
movies_df.dropna(subset=['directors'], inplace=True)
movies_df.reset_index(drop=True, inplace=True)
movies_df.shape

(6919, 12)

### Merge with Links Data<a id="mwld"></a>

In [ ]:
# Load links data.
links_df = pd.read_csv('links.csv')
print(links_df.shape)
links_df.head()

(45843, 3)


,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [ ]:
# Extract IDs based on the movies left in the movies_df.
links_df = links_df[links_df.tmdbId.isin(movies_df.id)]
links_df = links_df.drop_duplicates()
links_df.sort_values('tmdbId', inplace=True)
links_df.drop(links_df[links_df.tmdbId.duplicated(keep=False)].index[0::2], axis=0, inplace=True)
links_df.reset_index(drop=True, inplace=True)
print(links_df.shape)
links_df.head()

(6919, 3)


,movieId,imdbId,tmdbId
0,61724,92149,3.0
1,18,113101,5.0
2,479,107286,6.0
3,260,76759,11.0
4,6377,266543,12.0


In [ ]:
# Replace movies_df id with links_df movieId.
movies_df.id = links_df.movieId
movies_df.sort_values(by='id', ignore_index=True, inplace=True)
print(movies_df.shape)
movies_df.head()

(6919, 12)


,belongs_to_collection,genres,id,popularity,production_companies,release_date,runtime,title,vote_average,vote_count,directors,characters
0,Toy Story Collection,"[Animation, Comedy, Family]",1,21.946943,[Pixar Animation Studios],1995-10-30,81.0,Toy Story,7.7,5415.0,[John Lasseter],None
1,None,"[Adventure, Fantasy, Family]",2,17.015539,"[TriStar Pictures, Teitler Film, Interscope Co...",1995-12-15,104.0,Jumanji,6.9,2413.0,[Joe Johnston],"[Robin Williams, Jonathan Hyde, Kirsten Dunst]"
2,Grumpy Old Men Collection,"[Romance, Comedy]",3,11.712900,"[Warner Bros., Lancaster Gate]",1995-12-22,101.0,Grumpier Old Men,6.5,92.0,[Howard Deutch],"[Walter Matthau, Jack Lemmon, Ann-Margret]"
3,None,"[Comedy, Drama, Romance]",4,3.859495,[Twentieth Century Fox Film Corporation],1995-12-22,127.0,Waiting to Exhale,6.1,34.0,[Forest Whitaker],"[Whitney Houston, Angela Bassett, Loretta Devine]"
4,None,"[Action, Crime, Drama, Thriller]",6,17.924927,"[Regency Enterprises, Forward Pass, Warner Bros.]",1995-12-15,170.0,Heat,7.7,1886.0,[Michael Mann],"[Al Pacino, Robert De Niro, Val Kilmer]"


In [ ]:
# Export movies_df to movies_metadata_cleaned.csv file.
import os
newpath = r"dataset_cleaned"
if not os.path.exists(newpath):
    os.makedirs(newpath)
movies_df.to_csv(newpath + "/movies_metadata_cleaned.csv", index=False)

### One Hot Encoding Movies Metadata<a id="ohemm"></a>

In [ ]:
# One hot encoding all the collections and check the dimensions.
movies_ohe = movies_df.copy()
all_collections = pd.get_dummies(movies_ohe.belongs_to_collection)
all_collections.shape

(6919, 649)

In [ ]:
# Value counts for each collection.
all_collections.sum().sort_values(ascending=False)

James Bond Collection                  21
The Carry On Collection                12
The Fast and the Furious Collection     8
Harry Potter Collection                 8
Star Wars Collection                    8
                                       ..
Scanners Collection                     1
Scary Movie Collection                  1
Screamers Collection                    1
Sex and the City Collection             1
... Has Fallen Collection               1
Length: 649, dtype: int64

In [ ]:
# Select the collections that occured more than once and check the dimensions.
selected_collections = all_collections.loc[:, all_collections.sum() > 1]
selected_collections.shape

(6919, 275)

In [ ]:
# Append the one hot encoded collections to the movies_ohe.
movies_ohe.drop("belongs_to_collection", axis=1, inplace=True)
movies_ohe = pd.concat([movies_ohe, selected_collections], axis=1)
print(movies_ohe.shape)
movies_ohe.head()

(6919, 286)


,genres,id,popularity,production_companies,release_date,runtime,title,vote_average,vote_count,directors,characters,28 Days/Weeks Later Collection,300 Collection,A Better Tomorrow Collection,A Goofy Movie Collection,A Nightmare on Elm Street Collection,Ace Ventura Collection,Addams Family Collection,Afro Samurai Collection,Airplane Collection,Aladdin Collection,Alex Cross Collection,Alice in Wonderland Collection,Alien Collection,Alienation Trilogy,American Pie Collection,Anchorman Collection,Anne of Green Gables Collection,Arn collection,Asterix and Obelix (Animation) Collection,Austin Powers Collection,Babylon 5 Collection,Back to the Future Collection,Bad Boys Collection,Balto Collection,Bambi Collection,Barbershop Collection,Barrytown Trilogy,Batman (DC Universe Animated) Collection,Batman Beyond Collection,Batman Collection,Batman: The Dark Knight Returns Collection,Battlestar Galactica (Reboot) Collection,Before... Collection,Best Exotic Marigold Hotel Collection,Beverly Hills Cop Collection,Blade Collection,Bridget Jones Collection,Brooklyn Cigar Store Collection,Brother Bear Collection,Captain America Collection,Cars Collection,Che Collection,Cheech & Chong Collection,Cinderella Collection,Cinderella Story Collection,Class of 1999 Collection,Clerks Collection,Clint Eastwood's Iwo Jima Collection,Cloudy with a Chance of Meatballs Collection,Death Wish Collection,Despicable Me Collection,Diary of a Wimpy Kid Collection,Die Hard Collection,Dirty Dancing Collection,Dirty Harry Collection,Divergent Collection,Dolphin Tale Collection,Dr. Phibes Collection,Dr. Who Collection,Dracula (Hammer Series),Dracula (Universal Series),Dragon Ball Z (Movie) Collection,Elizabeth Collection,Europa Trilogy,Evil Dead Collection,Fantasia Collection,Fievel Collection,Finding Nemo Collection,Flowers in the Attic Collection,Frankenstein (Hammer Series),Frankenstein (Universal Series),French Connection Collection,Friday Collection,Friday the 13th Collection,Fright Night Collection,Futurama Collection,Ghostbusters Collection,Ginger Snaps Collection,Godzilla (Heisei) Collection,Goon Collection,Grindhouse Collection,Grumpy Old Men Collection,Guardians of the Galaxy Collection,Halloween Collection,Halloweentown Collection,Harold & Kumar Collection,Harry Palmer Collection,Harry Potter Collection,Heart of Gold Collection,Hellboy Collection,Hellraiser Collection,High School Musical Collection,Home Alone Collection,Honey Collection,Horrible Bosses Collection,Hot Shots! Collection,Hotel Transylvania Collection,House of 1000 Corpses Collection,How to Train Your Dragon Collection,Ice Age Collection,Indiana Jones Collection,Insidious Collection,Ip Man Collection,Iron Man Collection,Jackass Collection,James Bond Collection,Jesse Stone Collection,John Wick Collection,Johnny English Collection,Jump Street Collection,Jurassic Park Collection,Justice League (Animated) Collection,Kick-Ass Collection,Kidulthood Collection,Kill Bill Collection,Korengal Valley Documentaries,Kung Fu Panda Collection,LEGO DC Comics Super Heroes Collection,Lady and the Tramp Collection,Lethal Weapon Collection,Lew Harper Collection,Lilo & Stitch Collection,Living Dead Collection,Mad Max Collection,Madagascar Collection,Magic Mike Collection,Meet the Parents Collection,Men In Black Collection,Mexico Trilogy,Mickey's Upon a Christmas Collection,Mission: Impossible Collection,"Monsters, Inc. Collection",Mr. Bean Collection,Mulan Collection,Naked Gun Collection,Nanny McPhee Collection,National Lampoon's Vacation Collection,National Treasure Collection,Navarone Collection,Never Back Down Collection,Night at the Museum Collection,Night of the Demons Collection,Now You See Me Collection,Ocean's Collection,Once Upon a Time in China Collection,Peter Pan Collection,Phantasm Collection,Philo & Clyde Collection,Pirates of the Caribbean Collection,Pitch Perfect Collection,Planet of the Apes (Reboot) Collection,Planet of the Apes Original Collection,Pokémon Collection,Police Story Collection,Predator

In [ ]:
# One hot encoding all the genres and check the dimensions.
all_genres = movies_ohe.genres.str.join("|").str.get_dummies()
all_genres.shape

(6919, 20)

In [ ]:
# Value counts for each genre.
all_genres.sum().sort_values(ascending=False)

Drama              3952
Comedy             2147
Thriller           1662
Romance            1315
Action             1291
Crime              1111
Adventure           895
Horror              647
Family              614
Science Fiction     585
Mystery             572
Fantasy             490
Animation           389
History             355
Music               328
War                 299
Documentary         287
Western             193
TV Movie             70
Foreign              17
dtype: int64

In [ ]:
# Append the one hot encoded genres to the movies_ohe.
movies_ohe.drop("genres", axis=1, inplace=True)
movies_ohe = pd.concat([movies_ohe, all_genres], axis=1)
print(movies_ohe.shape)
movies_ohe.head()

(6919, 305)


,id,popularity,production_companies,release_date,runtime,title,vote_average,vote_count,directors,characters,28 Days/Weeks Later Collection,300 Collection,A Better Tomorrow Collection,A Goofy Movie Collection,A Nightmare on Elm Street Collection,Ace Ventura Collection,Addams Family Collection,Afro Samurai Collection,Airplane Collection,Aladdin Collection,Alex Cross Collection,Alice in Wonderland Collection,Alien Collection,Alienation Trilogy,American Pie Collection,Anchorman Collection,Anne of Green Gables Collection,Arn collection,Asterix and Obelix (Animation) Collection,Austin Powers Collection,Babylon 5 Collection,Back to the Future Collection,Bad Boys Collection,Balto Collection,Bambi Collection,Barbershop Collection,Barrytown Trilogy,Batman (DC Universe Animated) Collection,Batman Beyond Collection,Batman Collection,Batman: The Dark Knight Returns Collection,Battlestar Galactica (Reboot) Collection,Before... Collection,Best Exotic Marigold Hotel Collection,Beverly Hills Cop Collection,Blade Collection,Bridget Jones Collection,Brooklyn Cigar Store Collection,Brother Bear Collection,Captain America Collection,Cars Collection,Che Collection,Cheech & Chong Collection,Cinderella Collection,Cinderella Story Collection,Class of 1999 Collection,Clerks Collection,Clint Eastwood's Iwo Jima Collection,Cloudy with a Chance of Meatballs Collection,Death Wish Collection,Despicable Me Collection,Diary of a Wimpy Kid Collection,Die Hard Collection,Dirty Dancing Collection,Dirty Harry Collection,Divergent Collection,Dolphin Tale Collection,Dr. Phibes Collection,Dr. Who Collection,Dracula (Hammer Series),Dracula (Universal Series),Dragon Ball Z (Movie) Collection,Elizabeth Collection,Europa Trilogy,Evil Dead Collection,Fantasia Collection,Fievel Collection,Finding Nemo Collection,Flowers in the Attic Collection,Frankenstein (Hammer Series),Frankenstein (Universal Series),French Connection Collection,Friday Collection,Friday the 13th Collection,Fright Night Collection,Futurama Collection,Ghostbusters Collection,Ginger Snaps Collection,Godzilla (Heisei) Collection,Goon Collection,Grindhouse Collection,Grumpy Old Men Collection,Guardians of the Galaxy Collection,Halloween Collection,Halloweentown Collection,Harold & Kumar Collection,Harry Palmer Collection,Harry Potter Collection,Heart of Gold Collection,Hellboy Collection,Hellraiser Collection,High School Musical Collection,Home Alone Collection,Honey Collection,Horrible Bosses Collection,Hot Shots! Collection,Hotel Transylvania Collection,House of 1000 Corpses Collection,How to Train Your Dragon Collection,Ice Age Collection,Indiana Jones Collection,Insidious Collection,Ip Man Collection,Iron Man Collection,Jackass Collection,James Bond Collection,Jesse Stone Collection,John Wick Collection,Johnny English Collection,Jump Street Collection,Jurassic Park Collection,Justice League (Animated) Collection,Kick-Ass Collection,Kidulthood Collection,Kill Bill Collection,Korengal Valley Documentaries,Kung Fu Panda Collection,LEGO DC Comics Super Heroes Collection,Lady and the Tramp Collection,Lethal Weapon Collection,Lew Harper Collection,Lilo & Stitch Collection,Living Dead Collection,Mad Max Collection,Madagascar Collection,Magic Mike Collection,Meet the Parents Collection,Men In Black Collection,Mexico Trilogy,Mickey's Upon a Christmas Collection,Mission: Impossible Collection,"Monsters, Inc. Collection",Mr. Bean Collection,Mulan Collection,Naked Gun Collection,Nanny McPhee Collection,National Lampoon's Vacation Collection,National Treasure Collection,Navarone Collection,Never Back Down Collection,Night at the Museum Collection,Night of the Demons Collection,Now You See Me Collection,Ocean's Collection,Once Upon a Time in China Collection,Peter Pan Collection,Phantasm Collection,Philo & Clyde Collection,Pirates of the Caribbean Collection,Pitch Perfect Collection,Planet of the Apes (Reboot) Collection,Planet of the Apes Original Collection,Pokémon Collection,Police Story Collection,Predator Collec

In [ ]:
# One hot encoding all the production companies and check the dimensions.
all_companies = movies_ohe.production_companies.str.join("|").str.get_dummies()
all_companies.shape

(6919, 6861)

In [ ]:
# Value counts for each company.
all_companies.sum().sort_values(ascending=False)

Warner Bros.                                    462
Universal Pictures                              398
Paramount Pictures                              388
Twentieth Century Fox Film Corporation          319
Metro-Goldwyn-Mayer (MGM)                       218
                                               ... 
Motion Investment Group                           1
Motion Picture BETA Produktionsgesellschaft       1
Motion Picture JOTA Produktions                   1
Motion Picture KAPPA Produktionsgesellschaft      1
# Lexyn Productions                               1
Length: 6861, dtype: int64

In [ ]:
# Select the companies that occured at least ten times and check the dimensions.
selected_companies = all_companies.loc[:, all_companies.sum() >= 10]
selected_companies.shape

(6919, 224)

In [ ]:
# Append the one hot encoded companies to the movies_ohe.
movies_ohe.drop("production_companies", axis=1, inplace=True)
movies_ohe = pd.concat([movies_ohe, selected_companies], axis=1)
print(movies_ohe.shape)
movies_ohe.head()

(6919, 528)


,id,popularity,release_date,runtime,title,vote_average,vote_count,directors,characters,28 Days/Weeks Later Collection,300 Collection,A Better Tomorrow Collection,A Goofy Movie Collection,A Nightmare on Elm Street Collection,Ace Ventura Collection,Addams Family Collection,Afro Samurai Collection,Airplane Collection,Aladdin Collection,Alex Cross Collection,Alice in Wonderland Collection,Alien Collection,Alienation Trilogy,American Pie Collection,Anchorman Collection,Anne of Green Gables Collection,Arn collection,Asterix and Obelix (Animation) Collection,Austin Powers Collection,Babylon 5 Collection,Back to the Future Collection,Bad Boys Collection,Balto Collection,Bambi Collection,Barbershop Collection,Barrytown Trilogy,Batman (DC Universe Animated) Collection,Batman Beyond Collection,Batman Collection,Batman: The Dark Knight Returns Collection,Battlestar Galactica (Reboot) Collection,Before... Collection,Best Exotic Marigold Hotel Collection,Beverly Hills Cop Collection,Blade Collection,Bridget Jones Collection,Brooklyn Cigar Store Collection,Brother Bear Collection,Captain America Collection,Cars Collection,Che Collection,Cheech & Chong Collection,Cinderella Collection,Cinderella Story Collection,Class of 1999 Collection,Clerks Collection,Clint Eastwood's Iwo Jima Collection,Cloudy with a Chance of Meatballs Collection,Death Wish Collection,Despicable Me Collection,Diary of a Wimpy Kid Collection,Die Hard Collection,Dirty Dancing Collection,Dirty Harry Collection,Divergent Collection,Dolphin Tale Collection,Dr. Phibes Collection,Dr. Who Collection,Dracula (Hammer Series),Dracula (Universal Series),Dragon Ball Z (Movie) Collection,Elizabeth Collection,Europa Trilogy,Evil Dead Collection,Fantasia Collection,Fievel Collection,Finding Nemo Collection,Flowers in the Attic Collection,Frankenstein (Hammer Series),Frankenstein (Universal Series),French Connection Collection,Friday Collection,Friday the 13th Collection,Fright Night Collection,Futurama Collection,Ghostbusters Collection,Ginger Snaps Collection,Godzilla (Heisei) Collection,Goon Collection,Grindhouse Collection,Grumpy Old Men Collection,Guardians of the Galaxy Collection,Halloween Collection,Halloweentown Collection,Harold & Kumar Collection,Harry Palmer Collection,Harry Potter Collection,Heart of Gold Collection,Hellboy Collection,Hellraiser Collection,High School Musical Collection,Home Alone Collection,Honey Collection,Horrible Bosses Collection,Hot Shots! Collection,Hotel Transylvania Collection,House of 1000 Corpses Collection,How to Train Your Dragon Collection,Ice Age Collection,Indiana Jones Collection,Insidious Collection,Ip Man Collection,Iron Man Collection,Jackass Collection,James Bond Collection,Jesse Stone Collection,John Wick Collection,Johnny English Collection,Jump Street Collection,Jurassic Park Collection,Justice League (Animated) Collection,Kick-Ass Collection,Kidulthood Collection,Kill Bill Collection,Korengal Valley Documentaries,Kung Fu Panda Collection,LEGO DC Comics Super Heroes Collection,Lady and the Tramp Collection,Lethal Weapon Collection,Lew Harper Collection,Lilo & Stitch Collection,Living Dead Collection,Mad Max Collection,Madagascar Collection,Magic Mike Collection,Meet the Parents Collection,Men In Black Collection,Mexico Trilogy,Mickey's Upon a Christmas Collection,Mission: Impossible Collection,"Monsters, Inc. Collection",Mr. Bean Collection,Mulan Collection,Naked Gun Collection,Nanny McPhee Collection,National Lampoon's Vacation Collection,National Treasure Collection,Navarone Collection,Never Back Down Collection,Night at the Museum Collection,Night of the Demons Collection,Now You See Me Collection,Ocean's Collection,Once Upon a Time in China Collection,Peter Pan Collection,Phantasm Collection,Philo & Clyde Collection,Pirates of the Caribbean Collection,Pitch Perfect Collection,Planet of the Apes (Reboot) Collection,Planet of the Apes Original Collection,Pokémon Collection,Police Story Collection,Predator Collection,Prom Night Colle

In [ ]:
# One hot encoding all the directors and check the dimensions.
all_directors = movies_ohe.directors.str.join("|").str.get_dummies()
all_directors.shape

(6919, 3375)

In [ ]:
# Value counts for each director.
all_directors.sum().sort_values(ascending=False)

Woody Allen             43
Alfred Hitchcock        37
Clint Eastwood          30
Steven Spielberg        29
Martin Scorsese         26
                        ..
Liza Johnson             1
Liz Garbus               1
Liz Friedlander          1
Lisanne Pajot            1
A. Edward Sutherland     1
Length: 3375, dtype: int64

In [ ]:
# Select the directors that occured at least 10 times and check the dimensions.
selected_directors = all_directors.loc[:, all_directors.sum() >= 10]
selected_directors.shape

(6919, 91)

In [ ]:
# Append the one hot encoded directors to the movies_ohe.
movies_ohe.drop("directors", axis=1, inplace=True)
movies_ohe = pd.concat([movies_ohe, selected_directors], axis=1)
print(movies_ohe.shape)
movies_ohe.head()

(6919, 618)


,id,popularity,release_date,runtime,title,vote_average,vote_count,characters,28 Days/Weeks Later Collection,300 Collection,A Better Tomorrow Collection,A Goofy Movie Collection,A Nightmare on Elm Street Collection,Ace Ventura Collection,Addams Family Collection,Afro Samurai Collection,Airplane Collection,Aladdin Collection,Alex Cross Collection,Alice in Wonderland Collection,Alien Collection,Alienation Trilogy,American Pie Collection,Anchorman Collection,Anne of Green Gables Collection,Arn collection,Asterix and Obelix (Animation) Collection,Austin Powers Collection,Babylon 5 Collection,Back to the Future Collection,Bad Boys Collection,Balto Collection,Bambi Collection,Barbershop Collection,Barrytown Trilogy,Batman (DC Universe Animated) Collection,Batman Beyond Collection,Batman Collection,Batman: The Dark Knight Returns Collection,Battlestar Galactica (Reboot) Collection,Before... Collection,Best Exotic Marigold Hotel Collection,Beverly Hills Cop Collection,Blade Collection,Bridget Jones Collection,Brooklyn Cigar Store Collection,Brother Bear Collection,Captain America Collection,Cars Collection,Che Collection,Cheech & Chong Collection,Cinderella Collection,Cinderella Story Collection,Class of 1999 Collection,Clerks Collection,Clint Eastwood's Iwo Jima Collection,Cloudy with a Chance of Meatballs Collection,Death Wish Collection,Despicable Me Collection,Diary of a Wimpy Kid Collection,Die Hard Collection,Dirty Dancing Collection,Dirty Harry Collection,Divergent Collection,Dolphin Tale Collection,Dr. Phibes Collection,Dr. Who Collection,Dracula (Hammer Series),Dracula (Universal Series),Dragon Ball Z (Movie) Collection,Elizabeth Collection,Europa Trilogy,Evil Dead Collection,Fantasia Collection,Fievel Collection,Finding Nemo Collection,Flowers in the Attic Collection,Frankenstein (Hammer Series),Frankenstein (Universal Series),French Connection Collection,Friday Collection,Friday the 13th Collection,Fright Night Collection,Futurama Collection,Ghostbusters Collection,Ginger Snaps Collection,Godzilla (Heisei) Collection,Goon Collection,Grindhouse Collection,Grumpy Old Men Collection,Guardians of the Galaxy Collection,Halloween Collection,Halloweentown Collection,Harold & Kumar Collection,Harry Palmer Collection,Harry Potter Collection,Heart of Gold Collection,Hellboy Collection,Hellraiser Collection,High School Musical Collection,Home Alone Collection,Honey Collection,Horrible Bosses Collection,Hot Shots! Collection,Hotel Transylvania Collection,House of 1000 Corpses Collection,How to Train Your Dragon Collection,Ice Age Collection,Indiana Jones Collection,Insidious Collection,Ip Man Collection,Iron Man Collection,Jackass Collection,James Bond Collection,Jesse Stone Collection,John Wick Collection,Johnny English Collection,Jump Street Collection,Jurassic Park Collection,Justice League (Animated) Collection,Kick-Ass Collection,Kidulthood Collection,Kill Bill Collection,Korengal Valley Documentaries,Kung Fu Panda Collection,LEGO DC Comics Super Heroes Collection,Lady and the Tramp Collection,Lethal Weapon Collection,Lew Harper Collection,Lilo & Stitch Collection,Living Dead Collection,Mad Max Collection,Madagascar Collection,Magic Mike Collection,Meet the Parents Collection,Men In Black Collection,Mexico Trilogy,Mickey's Upon a Christmas Collection,Mission: Impossible Collection,"Monsters, Inc. Collection",Mr. Bean Collection,Mulan Collection,Naked Gun Collection,Nanny McPhee Collection,National Lampoon's Vacation Collection,National Treasure Collection,Navarone Collection,Never Back Down Collection,Night at the Museum Collection,Night of the Demons Collection,Now You See Me Collection,Ocean's Collection,Once Upon a Time in China Collection,Peter Pan Collection,Phantasm Collection,Philo & Clyde Collection,Pirates of the Caribbean Collection,Pitch Perfect Collection,Planet of the Apes (Reboot) Collection,Planet of the Apes Original Collection,Pokémon Collection,Police Story Collection,Predator Collection,Prom Night Collection,Psyc

In [ ]:
# One hot encoding all the characters and check the dimensions.
all_characters = movies_ohe.characters.str.join("|").str.get_dummies()
all_characters.shape

(6919, 7660)

In [ ]:
# Value counts for each character.
all_characters.sum().sort_values(ascending=False)

Robert De Niro    44
Michael Caine     42
Clint Eastwood    37
Meryl Streep      34
John Wayne        34
                  ..
Luc Roeg           1
Lucas Pittaway     1
Lucas Till         1
Luci Christian     1
\tRobert Osth      1
Length: 7660, dtype: int64

In [ ]:
# Select the characters that occured at least 10 times and check the dimensions.
selected_characters = all_characters.loc[:, all_characters.sum() >= 10]
selected_characters.shape

(6919, 375)

In [ ]:
# Append the one hot encoded characters to the movies_ohe.
movies_ohe.drop("characters", axis=1, inplace=True)
movies_ohe = pd.concat([movies_ohe, selected_characters], axis=1)
print(movies_ohe.shape)
movies_ohe.head()

(6919, 992)


,id,popularity,release_date,runtime,title,vote_average,vote_count,28 Days/Weeks Later Collection,300 Collection,A Better Tomorrow Collection,A Goofy Movie Collection,A Nightmare on Elm Street Collection,Ace Ventura Collection,Addams Family Collection,Afro Samurai Collection,Airplane Collection,Aladdin Collection,Alex Cross Collection,Alice in Wonderland Collection,Alien Collection,Alienation Trilogy,American Pie Collection,Anchorman Collection,Anne of Green Gables Collection,Arn collection,Asterix and Obelix (Animation) Collection,Austin Powers Collection,Babylon 5 Collection,Back to the Future Collection,Bad Boys Collection,Balto Collection,Bambi Collection,Barbershop Collection,Barrytown Trilogy,Batman (DC Universe Animated) Collection,Batman Beyond Collection,Batman Collection,Batman: The Dark Knight Returns Collection,Battlestar Galactica (Reboot) Collection,Before... Collection,Best Exotic Marigold Hotel Collection,Beverly Hills Cop Collection,Blade Collection,Bridget Jones Collection,Brooklyn Cigar Store Collection,Brother Bear Collection,Captain America Collection,Cars Collection,Che Collection,Cheech & Chong Collection,Cinderella Collection,Cinderella Story Collection,Class of 1999 Collection,Clerks Collection,Clint Eastwood's Iwo Jima Collection,Cloudy with a Chance of Meatballs Collection,Death Wish Collection,Despicable Me Collection,Diary of a Wimpy Kid Collection,Die Hard Collection,Dirty Dancing Collection,Dirty Harry Collection,Divergent Collection,Dolphin Tale Collection,Dr. Phibes Collection,Dr. Who Collection,Dracula (Hammer Series),Dracula (Universal Series),Dragon Ball Z (Movie) Collection,Elizabeth Collection,Europa Trilogy,Evil Dead Collection,Fantasia Collection,Fievel Collection,Finding Nemo Collection,Flowers in the Attic Collection,Frankenstein (Hammer Series),Frankenstein (Universal Series),French Connection Collection,Friday Collection,Friday the 13th Collection,Fright Night Collection,Futurama Collection,Ghostbusters Collection,Ginger Snaps Collection,Godzilla (Heisei) Collection,Goon Collection,Grindhouse Collection,Grumpy Old Men Collection,Guardians of the Galaxy Collection,Halloween Collection,Halloweentown Collection,Harold & Kumar Collection,Harry Palmer Collection,Harry Potter Collection,Heart of Gold Collection,Hellboy Collection,Hellraiser Collection,High School Musical Collection,Home Alone Collection,Honey Collection,Horrible Bosses Collection,Hot Shots! Collection,Hotel Transylvania Collection,House of 1000 Corpses Collection,How to Train Your Dragon Collection,Ice Age Collection,Indiana Jones Collection,Insidious Collection,Ip Man Collection,Iron Man Collection,Jackass Collection,James Bond Collection,Jesse Stone Collection,John Wick Collection,Johnny English Collection,Jump Street Collection,Jurassic Park Collection,Justice League (Animated) Collection,Kick-Ass Collection,Kidulthood Collection,Kill Bill Collection,Korengal Valley Documentaries,Kung Fu Panda Collection,LEGO DC Comics Super Heroes Collection,Lady and the Tramp Collection,Lethal Weapon Collection,Lew Harper Collection,Lilo & Stitch Collection,Living Dead Collection,Mad Max Collection,Madagascar Collection,Magic Mike Collection,Meet the Parents Collection,Men In Black Collection,Mexico Trilogy,Mickey's Upon a Christmas Collection,Mission: Impossible Collection,"Monsters, Inc. Collection",Mr. Bean Collection,Mulan Collection,Naked Gun Collection,Nanny McPhee Collection,National Lampoon's Vacation Collection,National Treasure Collection,Navarone Collection,Never Back Down Collection,Night at the Museum Collection,Night of the Demons Collection,Now You See Me Collection,Ocean's Collection,Once Upon a Time in China Collection,Peter Pan Collection,Phantasm Collection,Philo & Clyde Collection,Pirates of the Caribbean Collection,Pitch Perfect Collection,Planet of the Apes (Reboot) Collection,Planet of the Apes Original Collection,Pokémon Collection,Police Story Collection,Predator Collection,Prom Night Collection,Psycho Collecti

In [ ]:
# Export movies_ohe to movies_metadata_ohe.csv file.
newpath = r"dataset_cleaned"
if not os.path.exists(newpath):
    os.makedirs(newpath)
movies_ohe.to_csv(newpath + "/movies_metadata_ohe.csv", index=False)

## B. Ratings Data<a id="rd"></a>

### Clean Ratings Data<a id="crd"></a>

In [ ]:
# Load ratings data.
ratings_df = pd.read_csv('ratings.csv')
print(ratings_df.shape)
ratings_df.head()

(26024289, 4)


,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [ ]:
# Extract ratings based on the movies left in the movies_ohe and the ratings should be at least 3.5 (out of 5).
ratings_df = ratings_df[(ratings_df.movieId.isin(movies_ohe.id)) & (ratings_df.rating >= 3.5)]

# Sort ratings_df by userId and timestamp, then drop the rating and timestamp attributes.
ratings_df.sort_values(by=['userId', 'timestamp'], ignore_index=True, inplace=True)
ratings_df.drop(['rating', 'timestamp'], axis=1, inplace=True)
print(ratings_df.shape)
ratings_df.head()

(13751477, 2)


,userId,movieId
0,1,2762
1,1,54503
2,1,112552
3,1,96821
4,1,5577


### All Ratings Data<a id="ard"></a>

In [ ]:
# Group all movieId by userId for association rules mining
ratings_all = ratings_df.groupby("userId")["movieId"].apply(list).reset_index()
print(ratings_all.shape)
ratings_all.head()

(265662, 2)


,userId,movieId
0,1,"[2762, 54503, 112552, 96821, 5577, 92439, 4878..."
1,2,"[648, 260, 1356, 628, 1210, 1233, 605, 377, 339]"
2,3,"[527, 858]"
3,4,"[4014, 4896, 1221, 1097, 2023, 1197, 2706, 235..."
4,5,"[1230, 1302, 1270, 3079, 3114, 3156, 2707, 858..."


In [ ]:
# Export all ratings data to ratings_all.csv file
newpath = r"dataset_cleaned"
if not os.path.exists(newpath):
    os.makedirs(newpath)
ratings_all.to_csv(newpath + "/ratings_all.csv", index=False)

### Ratings Data Subsets<a id="rds"></a>

In [ ]:
# Insert movie ID into all_genres
all_genres.insert(0, "id", movies_ohe.id)
all_genres.head()

,id,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,Foreign,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,2,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,4,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0
4,6,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0


In [ ]:
# Group movieId by userId for each genre for association rules mining
ratings_subsets = {"Genre": list(), "Data": list()}
for cn in all_genres.columns[1:]:
    ID = all_genres.loc[all_genres[cn] == 1, "id"]
    subset = ratings_df[ratings_df.movieId.isin(ID)]
    data = subset.groupby("userId")["movieId"].apply(list).reset_index()
    ratings_subsets["Genre"].append(cn)
    ratings_subsets["Data"].append(data)
    print(cn + ":", data.shape)

Action: (225237, 2)
Adventure: (226229, 2)
Animation: (147742, 2)
Comedy: (236620, 2)
Crime: (221362, 2)
Documentary: (43869, 2)
Drama: (255259, 2)
Family: (184997, 2)
Fantasy: (190035, 2)
Foreign: (2188, 2)
History: (150453, 2)
Horror: (121771, 2)
Music: (121069, 2)
Mystery: (187006, 2)
Romance: (215145, 2)
Science Fiction: (202527, 2)
TV Movie: (4166, 2)
Thriller: (231447, 2)
War: (150520, 2)
Western: (89291, 2)


In [ ]:
# Export all ratings subsets to respective csv file
newpath = r"dataset_cleaned/ratings_subsets"
if not os.path.exists(newpath):
    os.makedirs(newpath)
for i in range(len(ratings_subsets["Genre"])):
    filename = "ratings_{}.csv".format(ratings_subsets["Genre"][i].replace(" ", "_").lower())
    ratings_subsets["Data"][i].to_csv(newpath + "/" + filename, index=False)